# Aggregating DataFrames in PySpark HW

First let's start up our PySpark instance

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('airbnb').getOrCreate()

## Read in the dataFrame for this Notebook

In [2]:
dataset_path = 'dataset/nyc_air_bnb.csv'
airbnb = spark.read.csv(dataset_path,header=True)

## About this dataset

This dataset describes the listing activity and metrics for Air BNB bookers in NYC, NY for 2019. Each line in the dataset is a booking. 

**Source:** https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data/data

Let's go ahead and view the first few records of the dataset so we know what we are working with.

In [3]:
airbnb.show(5)

+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|  id|                name|host_id|  host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|2539|Clean & quiet apt...|   2787|       John|           Brooklyn|   Kensington|40.64749|-73.97237|   Private room|  149|             1|                9| 2018-10-19|             0.21|                             6|             365|
|2595|Skylit Midtown Ca...|   2845|   Jennifer|          Manhatt

Now print the schema so we can make sure all the variables have the correct types

In [4]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: string (nullable = true)



Notice here that some of the columns that are obviously numeric have been incorrectly identified as "strings". Let's edit that. Otherwise we cannot aggregate any of the numeric columns.

In [5]:
import pyspark.sql.types as T
from pyspark.sql.functions import to_date
def change_data_type(df, col, new_type):
    return df.withColumn(col, df[col].cast(new_type))

airbnb = change_data_type(airbnb, 'id',  'int')
airbnb = change_data_type(airbnb, 'host_id', 'int')
airbnb = change_data_type(airbnb, 'latitude', 'float')
airbnb = change_data_type(airbnb,'longitude','float')
airbnb = change_data_type(airbnb, 'price', 'int')
airbnb = change_data_type(airbnb, 'minimum_nights', 'int')
airbnb = change_data_type(airbnb, 'number_of_reviews', 'int')
airbnb = change_data_type(airbnb,'reviews_per_month','float')
airbnb = change_data_type(airbnb, 'calculated_host_listings_count', 'int')
airbnb = change_data_type(airbnb, 'availability_365', 'int')
airbnb = airbnb.withColumn('last_review', to_date(airbnb["last_review"], format='MM/dd/yyyy'))
airbnb.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: date (nullable = true)
 |-- reviews_per_month: float (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)



### Alright now we are ready to dig in!


### 1. How many rows are in this dataset?

In [6]:
# Count the number of rows
num_rows = airbnb.count()

# Print the result
print(f"Number of rows in the dataset: {num_rows}")


Number of rows in the dataset: 49079


### 2. How many total reviews does each host have?

In [7]:
from pyspark.sql.functions import col, count

# Group the DataFrame by 'host_id' (assuming it uniquely identifies a host)
grouped_df = airbnb.groupBy("host_id")

# Calculate the total number of reviews for each host using 'count'
total_reviews_per_host = grouped_df.agg(count("number_of_reviews").alias("total_reviews"))

# Display the results
total_reviews_per_host.show()


+--------+-------------+
| host_id|total_reviews|
+--------+-------------+
|  291112|            1|
| 1384111|            2|
| 1597481|            1|
| 2108853|            1|
| 2429432|            1|
| 2530670|            1|
| 3432742|            1|
| 1360296|            1|
| 2124690|            1|
| 6414252|            1|
| 9637768|            1|
| 9947836|            2|
| 9430366|            1|
| 7974574|            1|
| 5907325|            1|
|13749425|            1|
| 5771331|            1|
| 9784206|            1|
| 4702135|            1|
|19239110|            1|
+--------+-------------+
only showing top 20 rows



### 3. Show the min and max of all the numeric variables in the dataset

In [11]:
from pyspark.sql.functions import col, min, max

# Filter numeric columns (assuming you know the column names)
numeric_cols = ["price", "minimum_nights", "number_of_reviews", "latitude", "longitude"]  # Replace with actual column names
numeric_df = airbnb.select(numeric_cols)
min_max_df = numeric_df.summary('min', 'max')

# Calculate min and max for each numeric column


# Display the results in a single row
min_max_df.show()


+-------+-----+--------------+-----------------+---------+-----------+
|summary|price|minimum_nights|number_of_reviews| latitude|  longitude|
+-------+-----+--------------+-----------------+---------+-----------+
|    min|  -74|             0|                0|-74.16254|  -74.24442|
|    max|10000|          1250|              629| 40.91306|2.4906404E7|
+-------+-----+--------------+-----------------+---------+-----------+



### 4. Which host had the highest number of reviews?

Only display the top result.

Bonus: format the column names

In [15]:
from pyspark.sql.functions import desc
sorted_number_of_reviews = airbnb.orderBy(desc(airbnb["number_of_reviews"]))
highest_num_of_reviews = sorted_number_of_reviews.first()
print(f'The host name with the highest number of reviews is: {highest_num_of_reviews["host_name"]}')


# Listing with the lowest price per night

The host name with the highest number of reviews is: Dona


### 5. On average, how many nights did most hosts specify for a minimum?

In [16]:
from pyspark.sql.functions import col, avg

# Group the DataFrame by 'host_id' (assuming it uniquely identifies a host)
grouped_df = airbnb.groupBy("host_id")

# Calculate the average minimum nights per host
avg_min_nights_per_host = grouped_df.agg(avg("minimum_nights").alias("avg_min_nights"))

# Display only the average value (assuming you're interested in the overall average)
avg_min_nights_per_host.select("avg_min_nights").show(1)  # Show only the first row


+--------------+
|avg_min_nights|
+--------------+
|           2.0|
+--------------+
only showing top 1 row



### 6. What is the most expensive neighborhood to stay in on average?

Note: only show the one result

In [24]:
from pyspark.sql.functions import col, avg, round

# Group the DataFrame by 'neighbourhood'
grouped_df = airbnb.groupBy("neighbourhood")

# Calculate the average price per listing (round to 2 decimal places for better readability)
avg_price_per_listing = grouped_df.agg(round(avg("price"), 2).alias("avg_price"))

# Sort by average price in descending order and get the first row (most expensive)
most_expensive_neighbourhood = avg_price_per_listing.orderBy("avg_price", ascending=False).first()

# Display the result (neighbourhood and average price)
print(f"The most expensive neighbourhood is: {most_expensive_neighbourhood.neighbourhood}")


The most expensive neighbourhood is: Fort Wadsworth


### 7. Display a two by two table that shows the average prices by room type (private and shared only) and neighborhood group (Manhattan and Brooklyn only)

In [25]:
from pyspark.sql.functions import col, avg, round

# Filter for private and shared room types (replace with actual names if different)
filtered_df = airbnb.filter(col("room_type").isin(["Private room", "Shared room"]))

# Filter for Manhattan and Brooklyn only (replace with actual names if different)
filtered_df = filtered_df.filter(
    col("neighbourhood_group").isin(["Manhattan", "Brooklyn"])
)

# Group by room type and neighbourhood group
grouped_df = filtered_df.groupBy(["room_type", "neighbourhood_group"])

# Calculate average price per listing (round to 2 decimal places)
avg_price_per_listing = grouped_df.agg(round(avg("price"), 2).alias("avg_price"))

# Display as a two-by-two table using PySpark SQL functions (assuming you have a way to display the results as a table)
avg_price_per_listing.crosstab("room_type", "neighbourhood_group").show()


+-----------------------------+--------+---------+
|room_type_neighbourhood_group|Brooklyn|Manhattan|
+-----------------------------+--------+---------+
|                  Shared room|       1|        1|
|                 Private room|       1|        1|
+-----------------------------+--------+---------+



### Alright that's all folks!

### Great job!